# Read the stitches

Read data produced by `2024-07_list_of_all_stitches_sorted_by_duration.ipynb` and the human annotations from the pigleg experiment Google spreadsheet.

In [12]:

%load_ext autoreload
%autoreload 
!pip install pandas==1.5.3 openpyxl


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
distutils: /home/appuser/.local/include/python3.8/UNKNOWN
sysconfig: /home/appuser/.local/include/python3.8
user = True
home = None
root = None
prefix = None
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [19]:

import numpy as np
import seaborn as sns
import scipy
import skimage

sns.set_style("ticks")
from pathlib import Path

import pandas as pd
import pigleg_evaluation_tools as pet

# Get the data 
fn = sorted(list(Path(".").glob("*list_of_all_stitches*.xlsx")))[-1]
print(fn.absolute())
dfall = pd.read_excel(fn)

#remove unnamed columns
dfall = dfall.loc[:, ~dfall.columns.str.contains('^Unnamed')]

dfall
 # list_of_all_stitches_sorted_by_duration.2025-02-14_21-51-27.xlsx

/webapps/piglegsurgery/devel/list_of_all_stitches_sorted_by_duration.2025-02-14_21-57-22.xlsx


,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Needle holder stitch heatmap score [%],AI movement evaluation stitch [%],Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],Needle holder stitch median area presence [%],...,Forceps curved piercing visibility [%],Forceps curved piercing length [m],Needle holder piercing heatmap score [%],Needle holder piercing heatmap l2 distance [-],Forceps piercing heatmap score [%],Forceps piercing heatmap l2 distance [-],Scissors piercing heatmap score [%],Scissors piercing heatmap l2 distance [-],stitch_id,done_by_expert
0,Pretest_Einzelknopfnaht_Temeschinko.Ina.mp4,NaN,NaN,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN,NaN,42.733333,13.866667,17.834395,93.630573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
1,Bildschirmaufnahme_2024-10-07_um_13.31.25.mov,NaN,NaN,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN,56.611203,199.066667,41.000000,63.800225,73.037927,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
2,pigleg_test_4.mp4,00:00:02 stitch_start\r\n00:00:04 stitch_end,1.0,http://suran.kky.zcu.cz:8100//uploader/web_rep...,NaN,79.405902,2.000000,2.000000,43.478261,60.869565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
3,EKN_10_CBA.mp4,NaN,NaN,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN,88.039617,31.333333,4.466667,62.946429,89.062500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True
4,EinzelknopfnahtGenoux-Jonas.mov,NaN,NaN,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN,81.318603,37.533333,0.400000,87.500000,89.375000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,IMG_1337.mov,Akkurate Stiche! Beim ersten Knoten verwendest...,5.0,http://suran.kky.zcu.cz//uploader/web_report/d...,NaN,76.646098,67.133333,290.066667,74.078341,59.907834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,False
357,IMG_3597.mov,NaN,NaN,http://suran.kky.zcu.cz//uploader/web_report/6...,NaN,83.429174,51.266667,298.600000,77.859238,80.938416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,False
358,Stresstest_nicht_gekürztes_Vid.webm,NaN,NaN,http://suran.kky.zcu.cz//uploader/web_report/c...,NaN,77.306725,59.800000,303.733333,73.282443,91.984733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,False
359,STresstest.webm,NaN,NaN,http://suran.kky.zcu.cz//uploader/web_report/b...,NaN,66.601286,84.866667,105.800000,0.000000,82.172373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,False


In [20]:
!pwd

/webapps/piglegsurgery/devel


In [21]:
import pigleg_evaluation_tools as pet
import pandas as pd
import json

with open("pigleg_spreadsheet_url.json", "r") as f:
    dct = json.load(f)
    url = dct["url"]

PIGLEG_SPREADSHEET_URL = url



# get list of sheet names
with pd.ExcelFile(url) as xls:
    sheet_names = xls.sheet_names
print(sheet_names)




['Algorithm', 'ANA New OSATS rating', 'CBA New OSATS rating', 'Carina', 'List 8', 'Mira', 'Ana', 'Ana OSATS', 'Prof. Dahmen', 'Carina OSATS', 'Comparison OSATS rating', 'annotators vs algorithm data ta', 'ann vs algorithm 2024-09', 'List 7', 'temp order', 'temp new AI values']


In [22]:
sheet_name = "annotators vs algorithm data ta"
header = 0
sheet_name = "CBA New OSATS rating"
header = 1
# name = "annotators vs algorithm data ta"
# header = 1
# name=8
# Read excel file from url, ignore first row, column names are in the second row
df_humans = pd.read_excel(url, sheet_name=sheet_name, header=header)
df_humans.columns

Index(['Unnamed: 0', 'filename', 'stitch_id', 'report_url', 'data_split',
       'Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5',
       ...
       'Forceps curved piercing length [pix]',
       'Forceps curved piercing visibility [s]',
       'Forceps curved piercing velocity',
       'Forceps curved piercing velocity std',
       'Forceps curved piercing velocity above threshold',
       'Forceps curved piercing velocity threshold [m/s]',
       'Forceps curved piercing unit',
       'Forceps curved piercing visibility [%]',
       'Forceps curved piercing length [m]', 'stitch_id.1'],
      dtype='object', length=262)

In [23]:
df_humans

,Unnamed: 0,filename,stitch_id,report_url,data_split,Item 1,Item 2,Item 3,Item 4,Item 5,...,Forceps curved piercing length [pix],Forceps curved piercing visibility [s],Forceps curved piercing velocity,Forceps curved piercing velocity std,Forceps curved piercing velocity above threshold,Forceps curved piercing velocity threshold [m/s],Forceps curved piercing unit,Forceps curved piercing visibility [%],Forceps curved piercing length [m],stitch_id.1
0,34.0,EinzelknopfnahtamSchweinefuss_Muenzner_Simeon.mp4,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,test,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,35.0,Bad-2021_11_03_A_Petersen_Insa_Einzelknopfnaht...,0.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,test,1.0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,36.0,Pre_-Test_Isabel_Linden.mp4,2.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,test,1.0,1.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,37.0,Medium-RPReplay_Final1634048018.mp4,0.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,test,0.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,38.0,Medium-2021_11_03_A_Seifert_Johanna_Einzelknop...,0.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,test,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,NaN,NaN,Surgery Analyser,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,Surgery Analyser,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,Surgery Analyser,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,Surgery Analyser,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:

# data frames can be merged by the file name and the stitch number
# We are looking what is missing in df_humans compared to dfall

dfall["file_stitch"] = dfall["filename"] + "_" + dfall["stitch_id"].astype(str)
df_humans["file_stitch"] = df_humans["filename"] + "_" + df_humans["stitch_id"].astype(str)
isin_both = dfall["file_stitch"].isin(df_humans["file_stitch"])
dfall["file_stitch"]


0        Pretest_Einzelknopfnaht_Temeschinko.Ina.mp4_0
1      Bildschirmaufnahme_2024-10-07_um_13.31.25.mov_0
2                                  pigleg_test_4.mp4_0
3                                     EKN_10_CBA.mp4_0
4                    EinzelknopfnahtGenoux-Jonas.mov_0
                            ...                       
356                                     IMG_1337.mov_4
357                                     IMG_3597.mov_4
358              Stresstest_nicht_gekürztes_Vid.webm_4
359                                  STresstest.webm_4
360                                   stresstest.MP4_4
Name: file_stitch, Length: 361, dtype: object

In [25]:
df_humans["file_stitch"]

0     EinzelknopfnahtamSchweinefuss_Muenzner_Simeon....
1     Bad-2021_11_03_A_Petersen_Insa_Einzelknopfnaht...
2                       Pre_-Test_Isabel_Linden.mp4_2.0
3               Medium-RPReplay_Final1634048018.mp4_0.0
4     Medium-2021_11_03_A_Seifert_Johanna_Einzelknop...
                            ...                        
77                                                  NaN
78                                                  NaN
79                                                  NaN
80                                                  NaN
81                                                  NaN
Name: file_stitch, Length: 82, dtype: object

In [26]:
#select what will be in the output
df_merged = pd.merge(
    # df_humans[["file_stitch", "Carina", "Ana", "Mira"]],
    df_humans[["file_stitch"]],
    dfall, on="file_stitch", how="outer", indicator=False)
df_merged



/opt/conda/lib/python3.8/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/opt/conda/lib/python3.8/site-packages/IPython/core/formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,file_stitch,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Needle holder stitch heatmap score [%],AI movement evaluation stitch [%],Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],...,Forceps curved piercing visibility [%],Forceps curved piercing length [m],Needle holder piercing heatmap score [%],Needle holder piercing heatmap l2 distance [-],Forceps piercing heatmap score [%],Forceps piercing heatmap l2 distance [-],Scissors piercing heatmap score [%],Scissors piercing heatmap l2 distance [-],stitch_id,done_by_expert
0,EinzelknopfnahtamSchweinefuss_Muenzner_Simeon....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bad-2021_11_03_A_Petersen_Insa_Einzelknopfnaht...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pre_-Test_Isabel_Linden.mp4_2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Medium-RPReplay_Final1634048018.mp4_0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medium-2021_11_03_A_Seifert_Johanna_Einzelknop...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,IMG_1337.mov_4,IMG_1337.mov,Akkurate Stiche! Beim ersten Knoten verwendest...,5.0,http://suran.kky.zcu.cz//uploader/web_report/d...,NaN,76.646098,67.133333,290.066667,74.078341,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,False
439,IMG_3597.mov_4,IMG_3597.mov,NaN,NaN,http://suran.kky.zcu.cz//uploader/web_report/6...,NaN,83.429174,51.266667,298.600000,77.859238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,False
440,Stresstest_nicht_gekürztes_Vid.webm_4,Stresstest_nicht_gekürztes_Vid.webm,NaN,NaN,http://suran.kky.zcu.cz//uploader/web_report/c...,NaN,77.306725,59.800000,303.733333,73.282443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,False
441,STresstest.webm_4,STresstest.webm,NaN,NaN,http://suran.kky.zcu.cz//uploader/web_report/b...,NaN,66.601286,84.866667,105.800000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,False


In [27]:

now_datetime_str = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
fn = f"all_stitches_with_human_annotations.{sheet_name.replace(' ','_')}.{now_datetime_str}.xlsx"
df_merged.to_excel(fn)

# Find missing human annotations

Compare the all stitches dataset with the human annotations dataset to find the missing annotations.

In [10]:


isin_both = dfall["file_stitch"].isin(df_humans["file_stitch"])


df_missing = dfall[isin_both == False][["filename", "stitch_id", "report_url", "movement evaluation stitch"]]

df_missing


,filename,stitch_id,report_url,movement evaluation stitch
0,Bad-2021_11_03_B_V_lz__Ann_Sophie_Einzelknopfn...,0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN
1,Bad-2021_11_03_A_Petersen_Insa_Einzelknopfnaht...,0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN
2,Bad-2021_11_03_A_Spitze_Vanessa_Einzelknopfnah...,0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,NaN
3,Good-2021_10_27_B_Etzel_Peter_Einzelknopfnaht_...,0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,3.5
4,Medium-2021_11_03_A_Tengler_Sophie_Einzelknopf...,0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,4.0
...,...,...,...,...
465,IMG_3597.mov,4,http://suran.kky.zcu.cz//uploader/web_report/6...,NaN
466,Stresstest_nicht_gekürztes_Vid.webm,4,http://suran.kky.zcu.cz//uploader/web_report/c...,NaN
467,STresstest.webm,4,http://suran.kky.zcu.cz//uploader/web_report/b...,NaN
468,stresstest.MP4,4,http://suran.kky.zcu.cz//uploader/web_report/4...,NaN


In [11]:
df_missing.to_excel("missing_human_annotations.xlsx")